# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries

import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import TruncatedSVD
import pickle
import nltk
nltk.download(['punkt', 'wordnet'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [11]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('disaster_messages3', con=engine)
df.head(3)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
X = df['message'] # Placing the message column
Y = df.drop(['message', 'genre', 'id', 'original'], axis = 1) # Holding the transformed categories' columns

In [4]:
Y.shape # Shape of the dataframe Y

(26206, 36)

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    
    ''' This function is created to perform actions like:
        lower() - lower casing the words
        strip() - Remove spaces before/after the token
        Normalize the tokens in the text and process through wordnetlemmaztizer
        Returns the tokenized and lemmatized tokens stored in clean_tokens'''
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    #clean_tokens = [lemmatizer.lemmatize(word) for word in tokens]
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    return clean_tokens    

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [13]:
# Pipeline1 created for the purpose of combining different tokenizers and classifier together for efficient model usage
pipeline = Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer()),
            ('clf', MultiOutputClassifier(RandomForestClassifier()))    
            ])

In [14]:
pipeline.get_params() # Parameters representing the pipeline features

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f0f2811f620>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
# Shape of the dataframe X
X.shape

(26206,)

In [9]:
X.shape[0] != Y.shape[0] # Line of code to check the shapes match

False

In [10]:
df.drop('related', axis=1, inplace=True)
df.head(3)

,id,message,original,genre,request,offer,aid_related,medical_help,medical_products,search_and_rescue,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,0,0,1,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
X = df['message']
Y = df.drop(['message', 'genre', 'id', 'original'], axis = 1)

In [16]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
# Splitting into train and test data for prediction and model score valuation
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 45)

In [18]:
# Fitting the train data to the classifier
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [19]:
def model_test(model, X_test, Y_test):
    '''
    Function to generate classification report on the model
    Input: Model, test set ie X_test & Y_test
    Output: Prints the Classification report'''
    
    y_pred = model.predict(X_test)
    for i, col in enumerate(Y_test):
        print(col)
        print(classification_report(Y_test[col], y_pred[:,i]))

In [20]:
model_test(pipeline, X_test, Y_test) # Evaluating the model's performance

related
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5241
          1       0.00      0.00      0.00         1

avg / total       1.00      1.00      1.00      5242

request
             precision    recall  f1-score   support

          0       0.89      0.99      0.93      4357
          1       0.84      0.39      0.53       885

avg / total       0.88      0.88      0.87      5242

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5224
          1       0.00      0.00      0.00        18

avg / total       0.99      1.00      0.99      5242

aid_related
             precision    recall  f1-score   support

          0       0.72      0.88      0.79      3054
          1       0.75      0.51      0.61      2188

avg / total       0.73      0.73      0.71      5242

medical_help
             precision    recall  f1-score   support

          0       0.92      1.00      0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


model_test(pipeline, X_test, Y_test)

### 6. Improve your model
Use grid search to find better parameters. 

In [24]:
# Involving paramters to make the search/processing better using GridSearch feature and storing it in CV.

parameters =  {'tfidf__use_idf': (True, False), 
              'clf__estimator__n_estimators': [50, 100], 
              'clf__estimator__min_samples_split': [2, 4]} 

cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=-1, verbose=2)

In [25]:
cv # Displating the features of the parameters for the model.

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'tfidf__use_idf': (True, False), 'clf__estimator__n_estimators': [50, 100], 'clf__estimator__min_samples_split': [2, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [26]:
# Testing the model with GridSearch generated paramters stored in cv.

cv.fit(X_train, Y_train)
model_test(cv, X_test, Y_test)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, tfidf__use_idf=True 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, tfidf__use_idf=True, total= 3.6min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, tfidf__use_idf=True 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  4.0min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, tfidf__use_idf=True, total= 3.6min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, tfidf__use_idf=True 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, tfidf__use_idf=True, total= 3.6min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, tfidf__use_idf=False 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, tfidf__use_idf=False, total= 3.6min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, tfidf__use_idf=False 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, tfidf__use_idf=False, total= 3.6min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, tfidf__use_idf=False 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, tfidf__use_idf=False, total= 3.6min
[CV] clf__estimator__min_samples_split=2, clf__estimato

[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed: 130.7min finished


related
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5241
          1       0.00      0.00      0.00         1

avg / total       1.00      1.00      1.00      5242

request
             precision    recall  f1-score   support

          0       0.89      0.99      0.94      4357
          1       0.89      0.41      0.56       885

avg / total       0.89      0.89      0.88      5242

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5224
          1       0.00      0.00      0.00        18

avg / total       0.99      1.00      0.99      5242

aid_related
             precision    recall  f1-score   support

          0       0.76      0.89      0.82      3054
          1       0.80      0.60      0.69      2188

avg / total       0.78      0.77      0.76      5242

medical_help
             precision    recall  f1-score   support

          0       0.92      1.00      0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [27]:
# Improve  the pipeline

# Adding new features apart from TF-IDF to new pipeline named pipeline2.

pipeline2 = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

In [28]:
pipeline2.get_params() # Viewing the parameters of the pipeline2

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=None, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
             learning_rate=1.0, n_estimators=50, random_state=None),
              n_jobs=1))],
 'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_range=(1, 1), preprocessor=None, stop

In [29]:
pipeline2.fit(X_train, Y_train) # Fitting the model to training data

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

In [ ]:
model_test(pipeline2, X_test, Y_test) # Evaluating the model's performance

In [30]:
parameters_adaboost = {
    'clf__estimator__learning_rate': [0.05],
    'clf__estimator__n_estimators': [100, 200, 300]
}

cv_adaboost = GridSearchCV(estimator=pipeline2, cv=2, n_jobs = 4, scoring='f1_weighted', verbose=3, param_grid=parameters_adaboost)

In [31]:
cv_adaboost.fit(X_train, Y_train)

Fitting 2 folds for each of 3 candidates, totalling 6 fits
[CV] clf__estimator__learning_rate=0.05, clf__estimator__n_estimators=100 
[CV] clf__estimator__learning_rate=0.05, clf__estimator__n_estimators=100 
[CV] clf__estimator__learning_rate=0.05, clf__estimator__n_estimators=200 
[CV] clf__estimator__learning_rate=0.05, clf__estimator__n_estimators=200 


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


[CV]  clf__estimator__learning_rate=0.05, clf__estimator__n_estimators=100, score=0.3439908343467924, total=14.1min
[CV] clf__estimator__learning_rate=0.05, clf__estimator__n_estimators=300 
[CV]  clf__estimator__learning_rate=0.05, clf__estimator__n_estimators=100, score=0.3381722014385752, total=14.3min
[CV] clf__estimator__learning_rate=0.05, clf__estimator__n_estimators=300 


[Parallel(n_jobs=4)]: Done   2 out of   6 | elapsed: 15.0min remaining: 30.1min
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


[CV]  clf__estimator__learning_rate=0.05, clf__estimator__n_estimators=200, score=0.40225444292189455, total=28.7min
[CV]  clf__estimator__learning_rate=0.05, clf__estimator__n_estimators=200, score=0.40970365318945334, total=28.7min


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


[CV]  clf__estimator__learning_rate=0.05, clf__estimator__n_estimators=300, score=0.4330446906480409, total=28.3min
[CV]  clf__estimator__learning_rate=0.05, clf__estimator__n_estimators=300, score=0.4410105970647745, total=28.2min


[Parallel(n_jobs=4)]: Done   6 out of   6 | elapsed: 44.2min finished


GridSearchCV(cv=2, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'clf__estimator__learning_rate': [0.05], 'clf__estimator__n_estimators': [100, 200, 300]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_weighted', verbose=3)

In [32]:
model_test(cv_adaboost, X_test, Y_test) # Testing the model using the classifier and test data.

related
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5241
          1       0.00      0.00      0.00         1

avg / total       1.00      1.00      1.00      5242

request
             precision    recall  f1-score   support

          0       0.89      0.98      0.94      4357
          1       0.83      0.42      0.55       885

avg / total       0.88      0.89      0.87      5242

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5224
          1       0.00      0.00      0.00        18

avg / total       0.99      1.00      0.99      5242

aid_related
             precision    recall  f1-score   support

          0       0.71      0.91      0.80      3054
          1       0.80      0.48      0.60      2188

avg / total       0.75      0.73      0.72      5242

medical_help
             precision    recall  f1-score   support

          0       0.92      1.00      0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [33]:
pickle.dump(cv_adaboost, open('classifier.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.